<a href="https://colab.research.google.com/github/amitjhirwal11/amitpro/blob/main/notebookbdcca21b18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
austinreese_craigslist_carstrucks_data_path = kagglehub.dataset_download('austinreese/craigslist-carstrucks-data')

print('Data source import complete.')


100%|██████████| 262M/262M [00:04<00:00, 57.2MB/s]

Extracting files...


Data source import complete.


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Data Cleaning

In [ ]:
df_path = '/kaggle/input/craigslist-carstrucks-data/'
files = os.listdir(df_path)
print(files)

In [ ]:
csv_f = 'vehicles.csv'
path = os.path.join(df_path, csv_f)
df = pd.read_csv(path)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()   # check missing values

In [ ]:
Missing_values = df.isnull()
for column in Missing_values.columns.values.tolist():
    print(Missing_values[column].value_counts())
    print("")

In [ ]:
numeric_columns = ['year', 'odometer', 'lat', 'long']
for column in numeric_columns:
    df[column] = df[column].fillna(df[column].mean())

In [ ]:
def fill_with_mode(column):
    if column.isna().all():
        return column.fillna('Unknown')
    else:
        mode_value = column.mode()[0]
        return column.fillna(mode_value)
columns_to_clean = categorical_columns = ['manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color']
for column in columns_to_clean:
    if column in df.columns:
        df[column].fillna(fill_with_mode(df[column]), inplace=True)

In [ ]:
df.drop(['region_url','image_url','description','county'],inplace=True,axis=1)

df['posting_date'] = df['posting_date'].fillna('Unknown')

In [ ]:
for col in df.columns:
    print( col,':', df[col].nunique())
    print(df[col].value_counts().nlargest(5))
    print('\n' + '-' * 20 + '\n')

In [ ]:
df.describe()

In [ ]:
df.describe(include = 'all')

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

In [ ]:
df.dtypes

In [ ]:
df['posting_date'] = pd.to_datetime(df['posting_date'], errors='coerce')

In [ ]:
df.head()

In [ ]:
df['cylinders'].replace(to_replace=['3 cylinders','4 cylinders','5 cylinders','6 cylinders','8 cylinders','10 cylinders','12 cylinders'
,'other'],value=[3,4,5,6,8,10,12,np.nan],inplace=True)

# EDA

## What is the most popular car type?

In [ ]:
df['type'].value_counts()

**We found sedan is the highest one.**

In [ ]:
car_type_counts = df['type'].value_counts().head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=car_type_counts.values, y=car_type_counts.index, palette='viridis')
plt.title('Top 10 Most Popular Car Types')
plt.xlabel('Number of Listings')
plt.ylabel('Car Type')
plt.show()

In [ ]:
df['type'].value_counts().head(5).plot(
    kind='pie',
    autopct='%1.1f%%',
    figsize=(8, 8),
    startangle=140,
    shadow=True,
    labels=df['type'].value_counts().head(5).index,  # Use specific labels
    colors=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0']  # Optional custom colors
)
plt.title('Top 5 Types')
plt.ylabel('')
plt.show()

## What is the distribution of cars according to title status?

In [ ]:
title_counts = df['title_status'].value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(x=title_counts.index, y=title_counts.values, palette='magma')
plt.title('Distribution of Cars by Title Status')
plt.xlabel('Title Status')
plt.ylabel('Number of Listings')
plt.show()


## What is the trend of data over time?

In [ ]:
df['posting_date'] = pd.to_datetime(df['posting_date'], utc=True)
df['year_month'] = df['posting_date'].dt.to_period('M')
trend = df['year_month'].value_counts().sort_index()
plt.figure(figsize=(12, 6))
trend.plot()
plt.title('Trend of Car Listings Over Time')
plt.xlabel('Year-Month')
plt.ylabel('Number of Listings')
plt.xticks(rotation=45)
plt.show()


In [ ]:
df['year'] = df['year'].astype(int)
yearly_distribution = df.groupby('year')['id'].count()
plt.figure(figsize=(14, 8))
yearly_distribution.plot(kind='area', color='green', alpha=0.4)
plt.title('Car Listings Over the Years')
plt.xlabel('Year')
plt.ylabel('Number of Cars')
plt.show()

## How does the price change over the years?

In [ ]:
price_by_year = df.groupby('year')['price'].mean().dropna()
plt.figure(figsize=(10, 6))
sns.lineplot(x=price_by_year.index, y=price_by_year.values, marker='o', color='b')
plt.title('Average Price of Cars Over Manufacturing Years')
plt.xlabel('Year')
plt.ylabel('Average Price')
plt.show()


## What is the most popular condition?

In [ ]:
df['condition'].value_counts()

**we can see that the highest is good.**

In [ ]:
condition_counts = df['condition'].value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(x=condition_counts.index, y=condition_counts.values, palette='coolwarm')
plt.title('Distribution of Car Conditions')
plt.xlabel('Condition')
plt.ylabel('Number of Listings')
plt.show()


## What is the relation between price and mileage?

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='odometer', y='price', data=df, alpha=0.5)
plt.title('Price vs Odometer')
plt.xlabel('Odometer (Miles)')
plt.ylabel('Price')
plt.show()


## What is the correlation between price, distance, and cylinders?

In [ ]:
corr = df[['price', 'odometer', 'cylinders']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap: Price, Odometer, Cylinders')
plt.show()


## How is price affected by fuel type?

In [ ]:
fuel_price = df.groupby('fuel')['price'].mean().sort_values()
plt.figure(figsize=(8, 5))
sns.barplot(x=fuel_price.index, y=fuel_price.values, palette='pastel')
plt.title('Average Price by Fuel Type')
plt.xlabel('Fuel Type')
plt.ylabel('Average Price')
plt.show()


## What is the average price based on manufacturer?

In [ ]:
manufacturer_price = df.groupby('manufacturer')['price'].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=manufacturer_price.values, y=manufacturer_price.index, palette='cubehelix')
plt.title('Top 10 Manufacturers with Highest Average Price')
plt.xlabel('Average Price')
plt.ylabel('Manufacturer')
plt.show()


## What is the distribution of cars based on cylinders?

In [ ]:
df['cylinders'].value_counts().sort_values()


In [ ]:
cylinder_types = df['cylinders'].dropna().unique()
cylinder_types

In [ ]:
plt.figure(figsize=(20, 20))
for i, cyl in enumerate(cylinder_types, 1):
    plt.subplot(len(cylinder_types), 1, i)
    sns.distplot(
        df[df['cylinders'] == cyl]['price'],
        color='maroon',
        hist_kws={'alpha': 0.7},
        kde_kws={'lw': 5}
    )
    plt.title(f'Price distribution of {cyl}-cylinder cars', y=-0.5)
    plt.xlabel('Price')
    plt.ylabel('Density')
plt.tight_layout()
plt.show()


In [ ]:
cylinder_counts = df['cylinders'].value_counts().dropna()
plt.figure(figsize=(8, 5))
sns.barplot(x=cylinder_counts.index, y=cylinder_counts.values, palette='viridis')
plt.title('Distribution of Cars by Cylinders')
plt.xlabel('Cylinders')
plt.ylabel('Number of Listings')
plt.show()


## Which state has the most car models in the USA?

In [ ]:
stat=df.groupby('state').count()['model'].sort_values(ascending=False).head()
stat

In [ ]:
state_counts = df['state'].value_counts().head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=state_counts.values, y=state_counts.index, palette='deep')
plt.title('Top 10 States with Most Car Listings')
plt.xlabel('Number of Listings')
plt.ylabel('State')
plt.show()


## Distribution on the map

In [ ]:
plt.figure(figsize = (20, 5))
sns.scatterplot(x=df['long'],y=df['lat'])
plt.title('Fig: Latitude and Longitude of each state', y=-0.23)

In [ ]:
sample_df = df.sample(int(0.01*len(df)))
lat_long_pairs = list(zip(list(df['lat']),list(df['long'])))

In [ ]:
map=folium.Map()
HeatMap(lat_long_pairs[50:150]).add_to(map)
map